In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import seaborn as sns
import os

class Gaussian:
    def __init__(self):
        self.x = np.linspace(-4, 4, 100)
        self.y = np.linspace(-4, 4, 100)
        self.X, self.Y = np.meshgrid(self.x, self.y)

        # Parameters for Langevin dynamics
        self.step_size = 0.01
        self.noise_std = 0.1
        self.n_steps = 200

        self.results_dir = 'gaussian_mcmc_results'
        os.makedirs(self.results_dir, exist_ok=True)

    def gaussian_energy(self, x, y):
        return (x**2 + y**2) / 2

    def animate_langevin_dynamics(self, n_particles=100):
        particles = np.random.randn(n_particles, 2) * 2

        fig, ax = plt.subplots(figsize=(10, 10))

        Z = self.gaussian_energy(self.X, self.Y)

        def update(frame):
            ax.clear()

            ax.contour(self.X, self.Y, Z, levels=20)

            nonlocal particles
            x, y = particles[:, 0], particles[:, 1]

            grad_x = x
            grad_y = y

            noise = np.sqrt(2 * self.step_size * self.noise_std) * np.random.randn(n_particles, 2)
            particles[:, 0] = x - self.step_size * grad_x + noise[:, 0]
            particles[:, 1] = y - self.step_size * grad_y + noise[:, 1]

            scatter = ax.scatter(particles[:, 0], particles[:, 1], c='red', alpha=0.6)

            ax.set_xlim(-4, 4)
            ax.set_ylim(-4, 4)
            ax.set_title(f'Langevin Dynamics - Step {frame}')

            return scatter,

        anim = FuncAnimation(fig, update, frames=self.n_steps,
                           interval=50, blit=True)

        writer = PillowWriter(fps=30)
        anim.save(f'{self.results_dir}/langevin_dynamics.gif', writer=writer)
        plt.close()

    def animate_trajectories(self, n_particles=10):

        particles = np.random.randn(n_particles, 2) * 2
        trajectories = [particles.copy()]

        # Setup the figure
        fig, ax = plt.subplots(figsize=(10, 10))

        Z = self.gaussian_energy(self.X, self.Y)

        colors = plt.cm.rainbow(np.linspace(0, 1, n_particles))

        def update(frame):
            ax.clear()

            ax.contour(self.X, self.Y, Z, levels=20, alpha=0.5)

            if frame >= len(trajectories):
                current = trajectories[-1]
                x, y = current[:, 0], current[:, 1]
                grad_x = x
                grad_y = y
                noise = np.sqrt(2 * self.step_size * self.noise_std) * np.random.randn(n_particles, 2)
                new_pos = np.zeros_like(current)
                new_pos[:, 0] = x - self.step_size * grad_x + noise[:, 0]
                new_pos[:, 1] = y - self.step_size * grad_y + noise[:, 1]
                trajectories.append(new_pos)

            for i in range(n_particles):
                trajectory = np.array(trajectories[:frame+1])
                ax.plot(trajectory[:, i, 0], trajectory[:, i, 1],
                       c=colors[i], alpha=0.5, label=f'Particle {i+1}')

                ax.scatter(trajectory[-1, i, 0], trajectory[-1, i, 1],
                         c=[colors[i]], s=100)

            ax.set_xlim(-4, 4)
            ax.set_ylim(-4, 4)
            ax.set_title(f'Particle Trajectories - Step {frame}')
            ax.legend()

            return ax.collections

        anim = FuncAnimation(fig, update, frames=self.n_steps,
                           interval=50, blit=True)

        writer = PillowWriter(fps=30)
        anim.save(f'{self.results_dir}/trajectories.gif', writer=writer)
        plt.close()

def main():
    viz = Gaussian()

    print("Langevin dynamics animation...")
    viz.animate_langevin_dynamics()

    print("Particle trajectories animation...")
    viz.animate_trajectories()

    print("Animations saved in 'gaussian_ebm_results' directory")

if __name__ == "__main__":
    main()

Langevin dynamics animation...
